# Funções de apoio

## Vamos criar algumas funções para não termos que reescrever códigos!

In [ ]:
#######################
## FUNÇÕES DE APOIO ###
#######################

from numpy import absolute
from numpy import std
from numpy import mean

 # Imprime a média dos resultados obtidos por todos os modelos testados
def print_results(prec, accur, mae, recall):
    print('Precision: %.3f (%.3f)' % (mean(prec), std(prec))) #buscamos 1
    print('Accuracy: %.3f (%.3f)' % (mean(accur), std(accur))) #buscamos 1
    print('Recall: %.3f (%.3f)' % (mean(recall), std(recall))) #buscamos 1
    print('MAE: %.3f (%.3f)' % (mean(absolute(mae)), std(absolute(mae)))) #buscamos o 0.0 de loss


# Imprime um gráfico com a matriz de confusão
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt

def plot_confusion_matrix(cm, title=None):
    cm_obj = ConfusionMatrixDisplay(cm, display_labels=['Ficou', 'Saiu'])
    fig, ax = plt.subplots(figsize=(5,5))
    cm_obj.plot(ax=ax)
    cm_obj.ax_.set(
                    title='Matriz Confusão' if title == None else title, 
                    xlabel='Predito', 
                    ylabel='Atual')
    

#Faz os testes com modelos e imprime os resultados de forma padronizada
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict

def test_model(X, y, model, plot_cmatrix=True, print_cmatrix=False, print_subtitles=False, title=None):
    # define the model cross-validation configuration
    cv = StratifiedKFold(n_splits=8, random_state=1, shuffle=True)
    scoring = ['accuracy', 'precision','neg_mean_absolute_error', 'recall']

    # define the data preparation and modeling pipeline
    pipeline = Pipeline(steps=[('m', model)])

    #Calcula os scores do kfold
    scores = cross_validate(pipeline, X, y, scoring=scoring, cv=cv, n_jobs=-1, return_estimator=True)

    #Recupera os dados do cross_validate
    prec = scores['test_precision']
    accur = scores['test_accuracy']
    mae = scores['test_neg_mean_absolute_error']
    recall = scores['test_recall']

    #Chama a função para imprimir os dados
    print_results(prec, accur, mae, recall)

    # Executa uma predição "crossvalidada"
    y_pred = cross_val_predict(pipeline, X, y, cv=cv)
    # Recupera a matriz confusão da validação cruzada
    cm = confusion_matrix(y_true=y, y_pred=y_pred)
    
    #Imprime o report do sklearn
    print(classification_report(y, y_pred, zero_division=0))

    #Imprime a legenda do que é apresentado no classification_report
    if (print_subtitles):
        print('Legenda:')
        print('*Precisão é a capacidade de um classificador de não rotular uma instância positiva que na verdade é negativa (melhor 1)\n\tTP/(TP + FP)')
        print('*Recall é a capacidade de um classificador de encontrar todas as instâncias positivas (melhor 1)\n\tTP/(TP + FN)')
        print('*F1 = Percentual de positivos corretos (melhor 1)\n\tF1 Score = 2*(Recall * Precision) / (Recall + Precision)')
        print('*Suporte é o número de ocorrências reais da classe no conjunto de dados especificado\n')
        print('*Macro avg é a média de precisão, recall e F1 de todas as classes\n\tmacro-avg = (precisão classe 0 + precisão classe 1)/2')
        print('*weighted avg é a média ponderada da precisão, recall e F1 de todas as classes mescladas\n\tweighted average = (TP de 0 + TP de 1)/(totais das classes 0 + 1)\n')


    #Imprime a matriz confusão como gráfico
    if(plot_cmatrix):
        plot_confusion_matrix(cm, title)
    
    #Imprime a matriz confusão como texto
    if(print_cmatrix):
        print(cm)

    # retorna os estimators treinados
    return scores['estimator']



# Função para transformar as colunas numéricas com MinMaxScaler e categóricas em OHE usando Pandas
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

def transform_columns(numerical, categorical, X_data):
    # Faz uma cópia do DataFrame para não alterar os dados originais
    df = X_data.copy()

    scaler = MinMaxScaler()
    # Coloca em escala os valores numericos usando Pandas (é do mesmo jeito que usando matrizes)
    df[numerical] = scaler.fit_transform(df[numerical])

    # Transforma as categoricas em OHE usando Pandas
    df = pd.get_dummies(df, dtype='int', columns=categorical)
    #Ajusta os nomes das colunas substituindo espaços por underscore
    df.columns = df.columns.str.replace(' ', '_')  

    return df 


# Função para retornar os nomes das colunas numéricas e categóricas a partir dos tipos de dados
def get_columns_types(X):
    numerical_ix = X.select_dtypes(include=['int64']).columns
    categorical_ix = X.select_dtypes(include=['object', 'category']).columns #Você pode incluir outros tipos nessa lista
    return numerical_ix, categorical_ix



# Função que ordena e plota um gráfico com a importância das variáveis de forma ordenada
def plot_sorted_importance(features, inportance):
    #Cria um dict
    out = {}
    for i in range(len(features)):
        out[features[i]] = inportance[i] 
    
    #Ordena o dic
    out = dict(sorted(out.items(), key=lambda item: item[1]))

    # #Imprime a importancia de cada variável
    # for k in out.keys():
    #     print('Feature: %s, Score: %.5f' % (k,out[k]))

    #Plota o gráfico de barras ordenado
    plt.bar([k for k in out.keys()], [out[k] for k in out.keys()])
    plt.xticks(rotation=90)
    plt.show()


# Função para avaliar os modelos com dados de validação
from sklearn.metrics import precision_score, accuracy_score, recall_score, mean_absolute_error

def validate_trained_models(estimators, X_valid, y_valid):
    #cria as listas vazias
    prec = []
    acc = []
    rec = []    
    mae = []

    # Percorre os estimators retornados pelo KFold
    for pipe in estimators:
        y_pred = pipe.named_steps['m'].predict(X_valid)
        prec.append(precision_score(y_valid, y_pred, zero_division=0))
        acc.append(accuracy_score(y_valid, y_pred))
        rec.append(recall_score(y_valid, y_pred))
        mae.append(mean_absolute_error(y_valid, y_pred))

    # Chama a função para imprimir os resultados
    print_results(prec, acc, mae, rec)

    #Gera a matriz confusão com o último modelo utilizado pelo KFold e exibe o gráfico
    cm = confusion_matrix(y_true=y_valid, y_pred=y_pred)
    plot_confusion_matrix(cm, title='Valores último modelo do KFold')

# Revisão

In [ ]:
import pandas as pd

#carregando o csv em um DataFrame usando a biblioteca Pandas
df = pd.read_csv('rh.csv')

## Exclui colunas desnecessárias
#-------------------------------
# As que possuem valor único
counts = df.nunique()
to_del = [df.columns[i] for i,v in enumerate(counts) if v == 1]
df.drop(to_del, axis=1, inplace=True)

#Exclui as que não fazem sentido para o negócio: com valor único para cada linha como contadores, ordenadores, identificadores etc. 
df.drop('codigo', axis=1, inplace=True)
#------------------------------

# Separa o X y do dataframe
X, y = df.drop('deixou_a_empresa', axis=1), df['deixou_a_empresa']

# Converte o y em numérico 0 ou 1 (vou fazer com Pandas o mesmo que o LabelEncoder faz)
y = y.map({'S':1, 'N':0})

# Como já conhecemos o conteúdo de nosso dataset (aula anterior)
# Vamos colocar em escala as colunas numéricas (MinMaxScaler) e transformar as categóricas em OHE (OneHotEncode)

# Primeiro recuperamos os nomes das colunas numericas e categóricas
numerical_ix, categorical_ix = get_columns_types(X)

# Agora usando função que criamos, vamos transformar os dados
X_matriz = transform_columns(numerical_ix, categorical_ix, X)
X_matriz.head()


In [ ]:
# Com nosso dataset preparado, vamos testar novamente o modelo de regressão logística

#Aplicando Regressão Logística com o dataframe  
from sklearn.linear_model import LogisticRegression

# Usando nossa função pra testar modelos com KFold
rls = test_model(X_matriz, 
           y,
           LogisticRegression(),
           print_cmatrix=True, 
           print_subtitles=True, 
           title='Regressão Logística')

In [ ]:
# Usando nossa função, vamos plotar a importancia das variáveis
plot_sorted_importance(X_matriz.columns, rls[0].named_steps['m'].coef_[0])

# 5 Live - Modelos e transformações (Continuação)

### Aplicando outros modelos nos mesmos dados

In [ ]:
#Aplicando RandomForest no mesmo dataframe para avaliar o modelo com os mesmos dados
from sklearn.ensemble import RandomForestClassifier

# Usando nossa função pra testar modelos com KFold
rfs = test_model(X_matriz, 
           y,
           RandomForestClassifier(),
           title='Random Forest')

In [ ]:
from sklearn.svm import SVC, LinearSVC

# Criando uma função para permitir os testes com SVM e KFold
def test_svms(X, y):

    # Vamos implementar vários parâmetros pra ver qual fica melhor
    models = (
        SVC(kernel="linear"),
        LinearSVC(max_iter=10000, dual="auto"),
        SVC(kernel="rbf", gamma=0.7),
        SVC(kernel="poly", degree=3, gamma="auto"),
    )

    titles = (
        "SVC with linear kernel",
        "LinearSVC (linear kernel)",
        "SVC with RBF kernel",
        "SVC with polynomial (degree 3) kernel",
    )

    estimators = []

    # Realizando o teste para cada modelo
    for t, model in zip(titles, models):
        print(t)
        # Recupero a lista de estimadores treinados
        estimators.append(test_model(X, y, model, title=t))
        print("")
    return estimators

In [ ]:
# Chamo a função para testar os svms com o dataframe
_ = test_svms(X_matriz, y)

### Undersampling

Nosso dataframe está desbalanceado. Alguns modelos são sensíveis a esse tipo de situação, porém outros ainda podem funcionar bem.

Nessa parte será necessário usar o `train_test_split` para separarmos um pacote de validação do modelo. O pacote de validação deve manter a proporção original de ocorrências para verificarmos se o novo viés afetou de sobremaneira o modelo.

In [ ]:
%pip install imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

#Retira uma parte dos dados para validação - com a configuracão original
X_rus, X_valid, y_rus, y_valid = train_test_split(X_matriz, y, test_size=0.20, random_state=33, stratify=y)

rus = RandomUnderSampler(random_state=0)

# Retira uma amostra randomica dos dados equilibrando x e y
X_res, y_res = rus.fit_resample(X_rus, y_rus)

print('Proporção antes')
print(y_rus.value_counts(), end='\n\n')

print('Nova proporção')
print(y_res.value_counts())

In [ ]:
# Agora vamos aplicar nos modelos e validar - Regressão Logística
lrs = test_model(X_res, y_res, LogisticRegression())


In [ ]:
# Testa os modelos gerados com os dados nas proporçoes reais que separamos para validação
validate_trained_models(lrs, X_valid, y_valid)

In [ ]:
#Treinando Random Forest balanceada
print("Resultado do Treino com dados balanceados")
rf = test_model(X_res, y_res, RandomForestClassifier())

In [ ]:

#Validando Random Forest
print("Resultado da Validação - Proporção original")
validate_trained_models(rf, X_valid, y_valid)

In [ ]:
#Treinando svms balanceados
print("Resultado do Treino com dados balanceados")
svms = test_svms(X_res, y_res)

In [ ]:

#Validando os svms
print("Resultado da Validação - Proporção original")

# Percorre os estimadores e faz o teste de validação
for i, estimators in enumerate(svms):
    print(f'Modelo {i}' )
    validate_trained_models(estimators, X_valid, y_valid)

## Selecionando as variáveis mais importantes

Nem sempre "`mais é melhor`" e muitas vezes variáveis redundantes podem gerar viézes que prejudicam os modelos. Portanto, nesse tópico vamos falar sobre seleção de variáveis pela importância.

In [ ]:
#Utilizaremos RFE para selecionar as variaveis mais importantes
from sklearn.feature_selection import RFE

# Aqui estamos imformando o número de variáveis que queremos utilizar
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=10)

rfe = rfe.fit(X_matriz, y)

#Colunas selecionadas
sel_cols = X_matriz.columns[rfe.support_]
print(sel_cols)

In [ ]:
# Utilizando as 10 colunas mais importantes como filtro, vamos testar o modelo de Regressao Logística!

_ = test_model(X_matriz[sel_cols], y, LogisticRegression())

In [ ]:
#Outra opção de utilização, é deixarmos o algoritmo selecionar automaticamente a quantidade ideal de 
#colunas para o modelo

from sklearn.feature_selection import RFECV

# Vamos criar uma função 
def get_rfe_cols(model):
    rfe = RFECV(estimator=model)

    rfe = rfe.fit(X_matriz, y)

    #Colunas selecionadas
    sel_cols = X_matriz.columns[rfe.support_]
    print(sel_cols)
    print("Quantidade selecionada: " + str(len(sel_cols)), end="\n\n")
    return sel_cols

In [ ]:
# Colunas mais importantes usando o modelo de Regressão Logística
sel_cols = get_rfe_cols(LogisticRegression())

_ = test_model(X_matriz[sel_cols], y, LogisticRegression())

In [ ]:
#Usando estimator em árvore
from sklearn.tree import DecisionTreeClassifier

#Chama a nossa função passando um estimador em forma de árvore!
sel_cols = get_rfe_cols(DecisionTreeClassifier())

#Testa o resultado com Random Forest
_ = test_model(X_matriz[sel_cols], y, RandomForestClassifier())

# Rode várias vezes e veja que cada tentativa pode gerar um resultado diferente!!

## Extratificação de variáveis - Variáveis numéricas em categóricas

Esse método deve ser usado somente quando existem outliers nos dados, distribuições multimodais, distribuições altamente exponenciais e outros casos que fogem muito da distribuição de probabilidade padrão.

In [ ]:
#Utilizando pandas.cut

# Copiando nosso dataframe original
X_extrat = X.copy()

classes = [0, 5000, 10000, 15000, 100000] #criando as classes
labels = ['até 5 mil', '5 a 10 mil', '10 a 15 mil', '+ de 15 mil'] #criando os labels

X_extrat['salario'] = pd.cut(X_extrat['salario'], classes, labels=labels)
X_extrat.head()

In [ ]:
# Agora vamos testar os resultados com a coluna salário em formato de categorias!
# Separo as columnas para transformacao
numerical_ix, categorical_ix = get_columns_types(X_extrat)

print(numerical_ix, categorical_ix)

# Transforma o dataframe para utilização nos modelos - chamamos nossa função
X_test = transform_columns(numerical_ix, categorical_ix, X_extrat)

# Listamos como ficou!
X_test.head()


In [ ]:
# Usando a função de teste, vejamos como fica aplicando Regressão Logística!
_ = test_model(X_test, y, LogisticRegression())

In [ ]:
# Usando a função de teste, vejamos como fica aplicando Random Forest!
_ = test_model(X_test, y, RandomForestClassifier())

In [ ]:
# Podemos também podemos extratificar colunas numéricas que são ordinais 
# para testar a influência delas nos modelos

# Convertemos o tipo da coluna de numérica para categórica (texto)
X_extrat['escolaridade'] = X_extrat['escolaridade'].apply(str)

X_extrat.dtypes

In [ ]:
# Repito os procedimentos usando nossas funcoes!!

#Recupero minhas colunas e transformo
numerical_ix, categorical_ix = get_columns_types(X_extrat)
X_test = transform_columns(numerical_ix, categorical_ix, X_extrat)

# Listamos como ficou!
print(X_test.columns)

In [ ]:
# Usando a função de teste, vejamos como fica aplicando Random Forest!
_ = test_model(X_test, y, RandomForestClassifier())

In [ ]:
# Usando a função de teste, vejamos como fica aplicando Regressão Logística!
_ = test_model(X_test, y, LogisticRegression())